# 데이터 불러오기

In [ ]:
df=pd.read_csv('../data/train_title_content_crawling.csv')

In [ ]:
len(df)

45678

# G2P 노이즈 데이터 제거

## `train_df의 text` == `크롤링한 제목` → 올바른 맞춤법

In [ ]:
data1=df[df['text']==df['crawling_title']].reset_index(drop=True)
data2=df[df['text']!=df['crawling_title']].reset_index(drop=True)

In [ ]:
print(len(data1))
print(len(data2))

6250
39428


In [ ]:
spell_df=data1.copy()[['ID', 'text', 'target', 'url', 'date', 'crawling_title']]
df=data2.copy()

## 숫자, 한글 이외 모든 문자를 제거 후 `train_df의 text` == `크롤링한 제목`

In [ ]:
df['text1']=df['text'].apply(lambda x: re.sub('[^0-9가-힣]','',x))
df['crawling_title1']=df['crawling_title'].apply(lambda x: re.sub('[^0-9가-힣]','',x))

In [ ]:
data1=df[df['text1']==df['crawling_title1']].reset_index(drop=True)
data2=df[df['text1']!=df['crawling_title1']].reset_index(drop=True)

In [ ]:
print(len(data1))
print(len(data2))

32968
6460


In [ ]:
spell_df=pd.concat([spell_df,data1[['ID', 'text', 'target', 'url', 'date', 'crawling_title']]]).reset_index(drop=True)
df=data2.copy().reset_index(drop=True)

In [ ]:
df['text1']=df['text1'].apply(lambda x: re.sub('종합[0-9]*[보]*','',x))
df['crawling_title1']=df['crawling_title1'].apply(lambda x: re.sub('종합[0-9]*[보]*','',x))

In [ ]:
data1=df[df['text1']==df['crawling_title1']].reset_index(drop=True)
data2=df[df['text1']!=df['crawling_title1']].reset_index(drop=True)

In [ ]:
print(len(data1))
print(len(data2))

154
6306


In [ ]:
spell_df=pd.concat([spell_df,data1[['ID', 'text', 'target', 'url', 'date', 'crawling_title']]]).reset_index(drop=True)
df=data2.copy().reset_index(drop=True)

## `train_data의 text`에 숫자가 포함 → 올바른 맞춤법

In [ ]:
df['IsNum']=df['text1'].apply(lambda x: 'Y' if len(re.findall('[0-9]',x))>0 else 'N')

In [ ]:
data1=df[df['IsNum']=='Y'].reset_index(drop=True)
data2=df[df['IsNum']=='N'].reset_index(drop=True)

In [ ]:
print(len(data1))
print(len(data2))

641
5665


## `크롤링 제목`에 `train_data의 text`가 포함 → 올바른 맞춤법

In [ ]:
for i in range(len(df)):
  if df.loc[i,'text1'] in df.loc[i,'crawling_title1']:
    df.loc[i,'IsContain']='Y'
  else:
    df.loc[i,'IsContain']='N'

In [ ]:
data1=df[df['IsContain']=='Y'].reset_index(drop=True)
data2=df[df['IsContain']=='N'].reset_index(drop=True)

In [ ]:
print(len(data1))
print(len(data2))

In [ ]:
spell_df=pd.concat([spell_df,data1[['ID', 'text', 'target', 'url', 'date', 'crawling_title']]]).reset_index(drop=True)
df=data2.copy().reset_index(drop=True)

# 올바른 맞춤법으로 필터링되지 않은 데이터는 `크롤링한 제목`으로 대체

In [ ]:
spell_df=pd.concat([spell_df,data1[['ID', 'text', 'target', 'url', 'date', 'crawling_title']]]).reset_index(drop=True)
df=data2.copy().reset_index(drop=True)

In [ ]:
df['text']=df['crawling_title'].apply(lambda x: re.sub('[,\"\'<>\[\]]','',x))

In [ ]:
spell_df=pd.concat([spell_df,df[['ID', 'text', 'target', 'url', 'date', 'crawling_title']]]).sort_values('ID').reset_index(drop=True)

In [ ]:
spell_df.to_csv('../data/train_spelling_v1.csv',encoding='utf-8-sig',index=False)